In [1]:
#!/usr/bin/env python3
# Imports
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from json import loads
from threading import Timer
from http.client import IncompleteRead
from tabulate import tabulate
import os

In [2]:
# token information, intialize this if empty or use runtime input
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""
users = {}
i = 0
duration = 60.0
keyword = ""

In [3]:
# StreamListener class to process Twitter stream using Tweepy Stream object
class StreamListener(StreamListener):
    '''StreamListener class: This class works as a handler for the incoming
    stream'''
    def on_data(self, data):
        '''on_data class_method: handle data by loading into a json and
        appending to global repository of tweets,
        based on minute as index'''
        global i, users
        tweet_json = loads(data)
        if 'user' in tweet_json:
            username = tweet_json['user']['screen_name']
            # increase count or add user to key and then increase count
            users[username] = users.get(username, 0) + 1
#             # debug print
#             if users[username] == 1:
#                 print("Minute {}: {} tweeted!".format(i, username))
#             else:
#                 print("Minute {}: {} tweeted AGAIN! Count = {}".format(i,
#                 username, users[username]))
        return True
    
    def on_error(self, status):
        '''on_error class_method: handle the errors are most frequently on
        server side/connection related and few, pass them'''
        pass

In [4]:
def printer():
    '''printer function: run a self-calling throad to be called every 60
    seconds on the clock (not equivalent to sleep!)'''
    global i, keyword, duration, users
    i = i + 1
    # called every minute; change value to 10 seconds to test faster
    Timer(duration, printer).start()
    if i != 1:
        print("\nUsers who tweeted about {} in last {} seconds:".format(keyword, duration))
        print(tabulate([(k, v) for k,v in users.items()] , headers=['Username', 'Tweets']))
        print("\n")
    # empty dictionary every 5 minutes
    if i % duration == 0:
        users = {}

In [ ]:
# MAIN FUNCTiON

# CHECK CREDENTIALS
if consumer_key == "" or consumer_secret == "" or access_token == "" or access_token_secret == "":
    print("Some values were not defined in the program! Create them at https://apps.twitter.com/")
if consumer_key == "":
    print("Enter consumer key:")
    consumer_key = input()
if consumer_secret == "":
    print("Enter consumer secret:")
    consumer_secret = input()
if access_token == "":
    print("Enter access token:")
    access_token = input()
if access_token_secret == "":
    print("Enter access token secret:")
    access_token_secret = input()
        
# TAKE INPUT
print("Enter your keyword:")
global keyword
keyword = input()

# COPIED
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# START PRINTING
print("\nConsuming the stream...")
# THIS WILL BECOME AN INDEPENDENT PROCESS
printer()

l = StreamListener()

# START STORING
while True:
    try:
        stream = Stream(auth, l)
        # THIS FUNCTION IS RUNNING AS `MAIN` PROCESS
        stream.filter(track=[keyword])
    except (Exception, IncompleteRead) as e:
        continue
        # kill kernel - need to address this
    except KeyboardInterrupt:
        os._exit(1)

Enter your keyword:
bitcoin

Consuming the stream...

Users who tweeted about bitcoin in last 10.0 seconds:
Username           Tweets
---------------  --------
BitcoinWrld             1
dtcz                    1
DenVichi                1
quotedian               1
Kuckel1                 1
about_today007          1
n1ktesla                1
BitCryptoX              1
cryptotroller           1
TempleRandazzo          1
mrfanterbulous          1
delicopsch56            1
RyoverlambOrac          1
BigHillDilligaf         1


